In [2]:
import random
from citipy import citipy
import requests
# Import the API key.
from config import weather_api_key
from datetime import datetime
import time
import pandas as pd

In [3]:
coordinates = set()
while len (coordinates) <2000:
    x = random.uniform(-90,90)
    y = random.uniform(-90,90)  
    pair = (x,y)
    coordinates.add(pair)


In [4]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)


753

In [5]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=2553670a2997579641ea610c5941e91e


In [6]:
# Create an endpoint URL for a city.
city_url = url + "&q=" + "Boston"
print(city_url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=2553670a2997579641ea610c5941e91e&q=Boston


In [7]:
city_weather = requests.get(city_url)
city_weather

<Response [200]>

In [8]:
# Get the JSON text of the 'Get' request.
city_weather.json()

{'coord': {'lon': -71.0598, 'lat': 42.3584},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 43.14,
  'feels_like': 37.96,
  'temp_min': 39.67,
  'temp_max': 46.17,
  'pressure': 1013,
  'humidity': 59},
 'visibility': 10000,
 'wind': {'speed': 8.99, 'deg': 267, 'gust': 20},
 'clouds': {'all': 75},
 'dt': 1642615318,
 'sys': {'type': 2,
  'id': 2013408,
  'country': 'US',
  'sunrise': 1642594114,
  'sunset': 1642628460},
 'timezone': -18000,
 'id': 4930956,
 'name': 'Boston',
 'cod': 200}

In [9]:
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1
# Loop through all the cities in our list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_description})
 # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | ostersund
Processing Record 2 of Set 1 | mirina
City not found. Skipping...
Processing Record 3 of Set 1 | benghazi
Processing Record 4 of Set 1 | arraial do cabo
Processing Record 5 of Set 1 | yirol
City not found. Skipping...
Processing Record 6 of Set 1 | laguna
Processing Record 7 of Set 1 | los llanos de aridane
Processing Record 8 of Set 1 | paamiut
Processing Record 9 of Set 1 | clyde river
Processing Record 10 of Set 1 | roald
Processing Record 11 of Set 1 | rabat
Processing Record 12 of Set 1 | georgetown
Processing Record 13 of Set 1 | east london
Processing Record 14 of Set 1 | oktyabrskoye
Processing Record 15 of Set 1 | torbay
Processing Record 16 of Set 1 | svetlogorsk
Processing Record 17 of Set 1 | victoria
Processing Record 18 of Set 1 | mahebourg
Processing Record 19 of Set 1 | jardim
Processing Record 20 of Set 1 | bredasdorp
Processing Record 21 of Set 1 | hamilton
Processing 

In [10]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df = city_data_df [["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]]
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Ostersund,SE,63.1792,14.6357,28.85,69,20,20.71,few clouds
1,Benghazi,LY,32.1167,20.0667,58.26,64,64,6.93,broken clouds
2,Arraial Do Cabo,BR,-22.9661,-42.0278,82.26,74,20,16.11,few clouds
3,Laguna,US,38.4210,-121.4238,59.50,85,0,1.01,clear sky
4,Los Llanos De Aridane,ES,28.6585,-17.9182,65.95,49,0,20.71,clear sky
5,Paamiut,GL,61.9940,-49.6678,37.81,82,100,49.19,moderate rain
6,Clyde River,CA,70.4692,-68.5914,5.23,85,100,37.98,heavy snow
7,Roald,NO,62.5833,6.1333,30.00,59,40,26.46,scattered clouds
8,Rabat,MA,33.9911,-6.8401,59.16,74,23,4.61,few clouds
9,Georgetown,MY,5.4112,100.3354,82.89,81,20,0.00,few clouds


In [11]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")